In [85]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import string
import itertools
import datetime

In [86]:
url = 'https://www.oddschecker.com/'

In [87]:
country_code = ['UK','IRE','USA','AUS']

In [117]:
def get_soup(base_url, sport = 'horses', event_url = None):
    '''Uses beautiful soup to get parse the url
    base_url = str, www.oddschecker.com/
    sport = str, which sport do you want to look at
    event_url = str, of the url extension which will take you to the '''
    
    if sport == 'horses':
        sport = 'horse-racing'
    
    url = base_url + sport
    if event_url != None:
        url += event_url
    
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    return soup(webpage, "html.parser")
    

In [118]:
page_soup = get_soup(url)


In [119]:
print(type(page_soup))

<class 'bs4.BeautifulSoup'>


In [120]:
def get_races(bsoup, country_codes, sport = 'horses'):
    '''Will return a dictionary of the events displayed on www.oddschecker.com
        Only does horse_racing atm.
        dict structure = events[countrycode][venue][list of event times]
        bsoup = the page parse with beautifulsoup4
        country_codes = countries you want to get events for
        sport = the sport you want''' 
    events = {code:{} for code in country_codes}
    containers = bsoup.findAll('div', {'class' : 'race-details'})
    for container in containers:
        txt = container.find('div', {'class' : 'venue-details'}).text
        
        for code in country_codes:
            # get country code and venue
            if code in txt[:3]:
                cc = code
                venue = txt.replace(code, '')
                break

        # get event times   
        times = [x.text for x in container.findAll('div', {'class' : 'racing-time'})]
        events[cc][venue] = times
        
    return events


In [121]:
events = get_races(page_soup, country_code)

In [122]:
class race():
    def __init__(self,base_url, sport, cc, venue, time):
        '''have a race as a class which we can add horse classes to.'''
        self.cc = cc
        self.venue = venue
        self.time = time
        self.url_ext = '/' + self.venue + '/' + self.time + '/' + 'winner'
        # soup the url
        soup = get_soup(base_url, sport, event_url = self.url_ext)
        
        # These containers are the rows in the table on the url
        containers = soup.findAll('tr', {'class' : 'diff-row evTabRow bc'})
#         print(len(containers))
        # init horse class
        self.horses = [horse(container) for container in containers] 
    
        
        
    def __str__(self):
        return f'{self.venue}, {self.cc} at {self.time}'
        
    def get_race_date(self,bsoup):
        '''placeholder for method to get the date of the race.  
        Will either be today or tomorrow on the website but haven''t worked it out how to grab it yet
        might work better in get races function'''
        pass
        
        

In [251]:
class horse():
    def __init__(self, container):
        ''''''
#         print(container)
        self.name = container.find('a', {'class' : 'popup selTxt'}).text
        self.jockey = container.find('div' ,{'class' :'bottom-row jockey'}).text
        print(self.name)
        print(self.jockey)
    def __str__(self,soup):
        pass
    

In [252]:
events['UK']

{'Huntingdon': ['13:40', '14:10', '14:45', '15:20', '15:55', '16:25', '16:55'],
 'Chester': ['13:50', '14:25', '15:00', '15:35', '16:05', '16:35', '17:05'],
 'Worcester': ['14:00', '14:35', '15:10', '15:45', '16:15', '16:45', '17:20'],
 'Wincanton': ['17:15', '17:50', '18:20', '18:50', '19:20', '19:50', '20:20'],
 'Chelmsford City': ['22:15', '22:45', '23:15', '23:45']}

In [253]:
events['UK'].keys()

dict_keys(['Huntingdon', 'Chester', 'Worcester', 'Wincanton', 'Chelmsford City'])

In [254]:
for (cc,v) in events.items():
    for venue, times in v.items():
        for time in times:
            x = race(url,'horses', cc, venue, time)            
            break
        break
    break

#         print(cc)
#         print(venue)
#         print(times)

Welsh Saint
Nico de Boinville421-
The Butcher Said
A P Heskin36/62-
Jackson Hill
R Johnson2/344-
Somekindofstar
L P Aspell2/F-
Mister Murchan
J M Davies/3P15-
Speedy Gonzalez
Brendan Powell40/38-


In [ ]:
# Cell to run 

# 1. get days races

# 2. Have a while loop running every X number of minutes until after the last race of the day

# 3. check time against time race starts, once it is Y number of hours before the start.  
#Start collecting odds data.  Odds data will get appended every X minutes as the while loop runs round

# 4.  re calculate odds stats (need to see what those are as haven't read the paper fully)

# 5. Highlight if we should bet based on betting strategy

# 6. Stop grabbing odds data once race has started.